In [1]:

#1.read data
import os
p_dir = "./dataset/modelnet40_normal_resampled"
classes = os.listdir(p_dir)
print("total classes ", classes)
print("number of classes ", len(classes))

#deprecated
def get_pcd_per_cls_deprecate(p_dir):
        classes = os.listdir(p_dir)
        for cls_path in classes:
            cls_path = os.path.join(p_dir, cls_path)
            pcd_per_cls = os.listdir(cls_path)[0]
            pcd_path = os.path.join(cls_path, pcd_per_class)
            print(pcd_path)
            
            
def get_key(p):
    return p.split("/")[0]


def get_pcd_cls(p_dir):
    m_ = {}
    listf = os.path.join(p_dir, "filelist.txt")
    with open(listf,"r") as f:
        l = f.readlines()
        l = list(map(lambda x:x.strip(), l))
        for i in l :
            key = get_key(i)
            if not key in m_:
                m_[key] = os.path.join(p_dir, i)
    return m_
    
m_ = get_pcd_cls(p_dir)
for k in m_:
    print("cls: ", k, " : ", m_[k])
print(len(m_))


total classes  ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'filelist.txt', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'modelnet10_shape_names.txt', 'modelnet10_test.txt', 'modelnet10_train.txt', 'modelnet40_shape_names.txt', 'modelnet40_test.txt', 'modelnet40_train.txt', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']
number of classes  47
cls:  airplane  :  ./dataset/modelnet40_normal_resampled/airplane/airplane_0001.txt
cls:  bathtub  :  ./dataset/modelnet40_normal_resampled/bathtub/bathtub_0001.txt
cls:  bed  :  ./dataset/modelnet40_normal_resampled/bed/bed_0001.txt
cls:  bench  :  ./dataset/modelnet40_normal_resampled/bench/bench_0001.txt
cls:  bookshelf  :  ./dataset/modelnet40_normal_resampled/bookshelf/bookshelf_0001.txt
c

In [2]:

def plot_pcd_from_path(path):
    pts = load_pcd(path)
    import open3d as o3d
    d_pcd = o3d.geometry.PointCloud()
    d_pcd.points = o3d.utility.Vector3dVector(pts[:,0:3])
    #d_pcd.colors = o3d.utility.Vector3dVector(pts[:,3:6])
    o3d.visualization.draw_geometries([d_pcd])

def plot_pcd_from_pts(pts):
    import open3d as o3d
    d_pcd = o3d.geometry.PointCloud()
    d_pcd.points = o3d.utility.Vector3dVector(pts[:,0:3])
    #d_pcd.colors = o3d.utility.Vector3dVector(pts[:,3:6])
    o3d.visualization.draw_geometries([d_pcd])
        
def load_pcd(path):
        import numpy as np
        pts = np.loadtxt(open(path,'r'), delimiter=',' , dtype = np.float32)
        pts = pts.reshape(-1, 6)
        return pts
        #d_pcd = o3d.geometry.PointCloud()
        #d_pcd.points = o3d.utility.Vector3dVector(pts[:,0:3])
        #d_pcd.colors = o3d.utility.Vector3dVector(pts[:,3:6])
        #o3d.visualization.draw_geometries([d_pcd])
        
        
def load_datas(dataset):
    res = {}
    for cls in dataset:
        path = dataset[cls]
        pts = load_pcd(path)
        res[cls] = pts
    return res
        
f = m_["airplane"]
plot_pcd_from_path(f)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# see airplane_origin.png

In [4]:
da = load_datas(m_)
da.keys()
#plot_pcd(da["airplane"])

dict_keys(['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox'])

In [7]:
import numpy as np
def pca(pts, number_pc= 1):
    print("raw shape",pts.shape)
    assert pts.shape[1] == 3
    X = pts.transpose()
    x_bar = np.sum(X, axis=1)/len(pts)
    print("x_bar",x_bar.transpose())
    print("X shape: ", X.shape)
    center_X = (X.transpose() - x_bar).transpose()
    center_X_t = center_X.transpose()
    print("x_bar_t",center_X_t.shape)
    H = np.matmul( center_X, center_X_t)
    print(H.shape)
    U,s, V = np.linalg.svd(H)
    target_eigen_vs = U[:,:number_pc]
    print(" u shape: ", U.shape)
    print("eigen values: ",s)
    print("target eigen vec shape: ", target_eigen_vs.shape)
    # encode phase
    proj_res = np.matmul(target_eigen_vs.transpose(), X)
    print("low dimension shape",proj_res.shape)
    reconv = np.matmul(target_eigen_vs, proj_res)
    print("reconv shape: ",reconv.shape)
    return reconv.transpose()
    #return pca_res.transpose()
    
for target in [ 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']:
    
   # target = "bottle"
    print(da[target].shape)
    air_plane_pcd = da[target][:,0:3]
    print(air_plane_pcd.shape)
    res = pca(air_plane_pcd, number_pc = 2)
    print(air_plane_pcd[:10,:])
    print(res[:10,:])

    #res2 = plot_pcd_from_pts(res)


(10000, 6)
(10000, 3)
raw shape (10000, 3)
x_bar [ 0.01076553  0.04789998 -0.02768866]
X shape:  (3, 10000)
x_bar_t (10000, 3)
(3, 3)
 u shape:  (3, 3)
eigen values:  [3309.6545   779.22943  451.39874]
target eigen vec shape:  (3, 2)
low dimension shape (2, 10000)
reconv shape:  (3, 10000)
[[ 0.7194 -0.2302 -0.211 ]
 [-0.874   0.3626  0.318 ]
 [-0.3837 -0.3652 -0.4021]
 [ 0.1239  0.3621  0.3195]
 [ 0.8739  0.3625  0.2468]
 [-0.8511 -0.3674  0.1797]
 [-0.3584  0.3691 -0.226 ]
 [-0.1195 -0.3009  0.3177]
 [ 0.4221  0.3375 -0.255 ]
 [ 0.4796 -0.1151  0.3206]]
[[ 0.71958643 -0.1984892  -0.23699284]
 [-0.8743404   0.3047007   0.3654591 ]
 [-0.38356695 -0.34255522 -0.42066154]
 [ 0.12354513  0.3017268   0.36898696]
 [ 0.8733188   0.2636084   0.32785994]
 [-0.8492729  -0.05655922 -0.07509144]
 [-0.36034155  0.03879543  0.04474564]
 [-0.11752408  0.03525068  0.04216238]
 [ 0.4201699   0.00913411  0.01415654]
 [ 0.48091874  0.10923629  0.13671495]]
(10000, 6)
(10000, 3)
raw shape (10000, 3)
x_ba

In [8]:
def normal_estimation(pts):
    print("shape of pts: ", pts.shape)
    
    pts = pts[:,0:3]

def plot_pcd_with_normal(pts):
    print("shape of pts: ", pts.shape)
    
    import open3d as o3d
    d_pcd = o3d.geometry.PointCloud()
    d_pcd.points = o3d.utility.Vector3dVector(pts[:,0:3])
    d_pcd.normals = o3d.utility.Vector3dVector(pts[:,3:6])
    #d_pcd.colors = o3d.utility.Vector3dVector(pts[:,3:6])
    o3d.visualization.draw_geometries([d_pcd],point_show_normal=True)
    
airplane_p = m_["airplane"]
pts = load_pcd(airplane_p)
normal_estimation(pts)
plot_pcd_with_normal(pts)

shape of pts:  (10000, 6)
shape of pts:  (10000, 6)
